# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Append project path

In [ ]:
import sys

PROJECT_PATH_UTIL = "/content/drive/MyDrive/Colab Notebooks/SCVQA/util"
#                    /content/drive/MyDrive/.../util
sys.path.append(PROJECT_PATH_UTIL)

# Install requirements

In [ ]:
!pip install scikit-video

# Import libraries and py script coded

In [ ]:
import torch

import pandas as pd
import numpy as np

import datetime
from timeit import default_timer as timer
from tqdm.auto import tqdm

import zipfile
from pathlib import Path
import os
import shutil

from dataset import VideoDataset
from model import ResNet50

# Setup parameters

In [ ]:
DATABASE = "CSCVQ"
CNN_MODULE = "ResNet50"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
FRAME_BATCH_SIZE = 32
MAX_FRAME_SIZE = 300

VIDEO_HEIGHT = 720
VIDEO_WIDTH = 1280


SOURCE_DIR = Path(f"/content/drive/MyDrive/Colab Notebooks/SCVQA/source/{DATABASE}/")
#                   /content/drive/MyDrive/.../source/{DATABASE}/
SOURCE_VIDEO_DIR = Path(f"/content/drive/MyDrive/Colab Notebooks/SCVQA/source/{DATABASE}/videos/")
#                         /content/drive/MyDrive/.../source/{DATABASE}/videos/
SOURCE_VIDEO_MOS_FILE = Path(f"/content/drive/MyDrive/Colab Notebooks/SCVQA/source/{DATABASE}/CSCVQ1.0-MOS.xlsx")
#                              /content/drive/MyDrive/.../source/{DATABASE}/CSCVQ1.0-MOS.xlsx

DATA_DIR = Path(f"/content/drive/MyDrive/Colab Notebooks/SCVQA/data/{DATABASE}/")
#                 /content/drive/MyDrive/.../data/{DATABASE}/
DATA_VIDEO_DIR = Path(f"/content/drive/MyDrive/Colab Notebooks/SCVQA/data/{DATABASE}/videos/")
#                       /content/drive/MyDrive/.../data/{DATABASE}/videos/
DATA_VIDEO_MOS_FILE = Path(f"/content/drive/MyDrive/Colab Notebooks/SCVQA/data/{DATABASE}/CSCVQ1.0-MOS.xlsx")
#                            /content/drive/MyDrive/.../data/{DATABASE}/CSCVQ1.0-MOS.xlsx

FEATURE_DIR = Path(f"/content/drive/MyDrive/Colab Notebooks/SCVQA/feature/{DATABASE}/{CNN_MODULE}/")
#                    /content/drive/MyDrive/.../feature/{DATABASE}/{CNN_MODULE}/

print(
    f"database: {DATABASE}, CNN module: {CNN_MODULE}, device: {DEVICE}, frame_batch_size: {FRAME_BATCH_SIZE}"
)

if not os.path.exists(FEATURE_DIR):
    FEATURE_DIR.mkdir(parents=True, exist_ok=True)

if not os.path.exists(DATA_DIR) and not os.path.exists(SOURCE_DIR):
    print(f"No source found to be extracted in {SOURCE_DIR}")
    sys.exit()

database: CSCVQ, CNN module: ResNet50, device: cuda, frame_batch_size: 32


# Extract video data and score data

In [ ]:
# Video
if os.path.exists(DATA_VIDEO_DIR):
    print(f"Video data exists in {DATA_VIDEO_DIR}/")
else:
    DATA_VIDEO_DIR.mkdir(parents=True, exist_ok=True)
    for dir_path, dir_names, file_names in os.walk(SOURCE_VIDEO_DIR):
        for file_name in file_names:
            file_path = os.path.join(dir_path, file_name)
            with zipfile.ZipFile(file_path, "r") as zip_ref:
                print(f"Unzipping {DATABASE}: {file_path}")
                zip_ref.extractall(DATA_VIDEO_DIR)

# MOS
if os.path.exists(DATA_VIDEO_MOS_FILE):
    print(f"MOS data exists in {DATA_VIDEO_MOS_FILE}")
else:
    print(f"Copying {DATABASE} MOS: {SOURCE_VIDEO_MOS_FILE}")
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    shutil.copyfile(SOURCE_VIDEO_MOS_FILE, DATA_VIDEO_MOS_FILE)

print(f"{DATABASE} data in: {DATA_DIR}")

Video data exists in /content/drive/MyDrive/Colab Notebooks/SCVQA/data/CSCVQ/videos/
MOS data exists in /content/drive/MyDrive/Colab Notebooks/SCVQA/data/CSCVQ/CSCVQ1.0-MOS.xlsx
CSCVQ data in: /content/drive/MyDrive/Colab Notebooks/SCVQA/data/CSCVQ


# Prepare extraction data

In [ ]:
dataset_df = pd.read_excel(str(DATA_VIDEO_MOS_FILE), header=None)
dataset_df = dataset_df[:-1]
# Delete last row that contains invalid label

dataset = VideoDataset(
    video_dir=str(DATA_VIDEO_DIR),
    height=VIDEO_HEIGHT,
    width=VIDEO_WIDTH,
    dataset_df=dataset_df,
    max_frame_size=MAX_FRAME_SIZE,
)
print(f"Number of video data to be extracted: {len(dataset)}")

Number of video data to be extracted: 165


# Extract CNN features

In [ ]:
start_time = timer()

resnet50 = ResNet50().to(device=DEVICE)
resnet50.eval()
with torch.inference_mode():
    for i in tqdm(range(len(dataset))):
        video_name, _ = dataset.get_video_info(i)

        OUTPUT_DIR = Path(f"/content/drive/MyDrive/Colab Notebooks/SCVQA/feature/{DATABASE}/{CNN_MODULE}/{video_name}")
        #                   /content/drive/MyDrive/.../feature/{DATABASE}/{CNN_MODULE}/{video_name}
        if os.path.exists(OUTPUT_DIR):
          continue

        video, mos = dataset[i]

        current = 0
        end_frame = len(video)

        video = video.to(device=DEVICE)

        feature_mean = torch.Tensor().to(device=DEVICE)
        feature_std = torch.Tensor().to(device=DEVICE)
        cnn_feature = torch.Tensor().to(device=DEVICE)

        while current < end_frame:
            head = current
            tail = (
                (head + FRAME_BATCH_SIZE)
                if (head + FRAME_BATCH_SIZE < end_frame)
                else end_frame
            )
            print(f"Extracting {video_name}: index[{i}] | frames[{head}, {tail-1}]")
            batch_frames = video[head:tail]

            mean, std = resnet50(batch_frames)
            feature_mean = torch.cat((feature_mean, mean), 0)
            feature_std = torch.cat((feature_std, std), 0)

            current += FRAME_BATCH_SIZE

        cnn_feature = (
            torch.cat((feature_mean, feature_std), 1).squeeze().numpy(force=True)
            if torch.cuda.is_available()
            else torch.cat((feature_mean, feature_std), 1).squeeze().numpy()
        )

        if not os.path.exists(OUTPUT_DIR):
            OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
            np.save(f"{OUTPUT_DIR}/feature", cnn_feature)
            np.save(f"{OUTPUT_DIR}/mos", mos)
end_time = timer()
print(
    f"Total extraction time: {datetime.timedelta(seconds=int(end_time-start_time))} (Hour:Minute:Second)"
)

  0%|          | 0/165 [00:00<?, ?it/s]

Extracting sc_slideShow_1280x720_30_8bit_420_300f_264QP30.yuv: index[151] | frames[0, 31]
Extracting sc_slideShow_1280x720_30_8bit_420_300f_264QP30.yuv: index[151] | frames[32, 63]
Extracting sc_slideShow_1280x720_30_8bit_420_300f_264QP30.yuv: index[151] | frames[64, 95]
Extracting sc_slideShow_1280x720_30_8bit_420_300f_264QP30.yuv: index[151] | frames[96, 127]
Extracting sc_slideShow_1280x720_30_8bit_420_300f_264QP30.yuv: index[151] | frames[128, 159]
Extracting sc_slideShow_1280x720_30_8bit_420_300f_264QP30.yuv: index[151] | frames[160, 191]
Extracting sc_slideShow_1280x720_30_8bit_420_300f_264QP30.yuv: index[151] | frames[192, 223]
Extracting sc_slideShow_1280x720_30_8bit_420_300f_264QP30.yuv: index[151] | frames[224, 255]
Extracting sc_slideShow_1280x720_30_8bit_420_300f_264QP30.yuv: index[151] | frames[256, 287]
Extracting sc_slideShow_1280x720_30_8bit_420_300f_264QP30.yuv: index[151] | frames[288, 299]
Extracting sc_slideShow_1280x720_30_8bit_420_300f_264QP36.yuv: index[152] | fr